# Province Share Modeling (Synthetic Demo)

This notebook is cleaned for GitHub and uses **synthetic sample data**.

## How to run
1) `pip install -r requirements.txt`
2) Run all cells


In [ ]:
import pandas as pd

DATA_PATH = "../examples/sample_province_share.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"]) 
df.head()

## Project: States Analysis v4

# final version. run THIS

 This Notebook was maintained to predict each states' portion of total order Order for each day


<sub>Customer Insights @ Digikala</sub>

<sub>10 June 2024</sub>



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import json

import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from xgboost import XGBRegressor
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
data_file = r'/content/dataaa_result_danesh_ready_for_add_oct13.xlsx'

#sheet = 'Danesh p'
#sheet = 'Shadabad p'

data = pd.read_excel(data_file)

data.tail(60)

In [ ]:
data.info()

In [ ]:
data.drop(columns=['Unnamed: 0_y','Unnamed: 0_x'],inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
output = pd.DataFrame()

In [ ]:
data=data[data['PersianDate']>=14010101]

In [ ]:
tempp=data[data['oct'].isnull()]
temppp=tempp[tempp['PersianDate']<14030101 ]

In [ ]:
temppp

In [ ]:
temppp['state_name_en'].unique()

In [ ]:
for city in temppp['state_name_en'].unique():
    bin = data[data['state_name_en'] == city].copy()  # Create a copy to avoid SettingWithCopyWarning
    bin['oct'].fillna(bin['oct'].mean(), inplace=True)  # Fill NaN values
    data.loc[data['state_name_en'] == city, 'oct'] = bin['oct']  # Update the main DataFrame


In [ ]:
data[data['PersianDate']<=14030930].isnull().sum()

In [ ]:
output = pd.DataFrame()

In [ ]:
data.drop(columns=['trend','pct_change','dollar','plus_sub_sum_past_30days','plus_subscription'],inplace=True)

In [ ]:
prov=data['State'].unique()
def objective(trial):

  reg = ExtraTreesRegressor(max_depth=trial.suggest_int("max_depth", 50, 100))
  reg.fit(X_train, y_train)
  y_pred = reg.predict(X_test)
  return mean_absolute_error(y_test, y_pred)

In [ ]:
train_data=data.iloc[:(-2*31*30)+31*30]

In [ ]:
data.iloc[-2*31*30+31*29:]

In [ ]:
output=pd.DataFrame()
#train_data = data.iloc[365*31:-4*31*30]
for p in prov:
    print(f'prov is  :  {p}')
    globals()[p] = train_data[train_data['State'] == p]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        globals()[p].drop(columns=['portion', 'state_name_en', 'State','order_count']),
        globals()[p]['portion'], test_size=0.2, random_state=42)

    print(f'thiiiiiiiiiiiiiiiiiiiiiiiis {type(X_train)}')

    # Setup Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=15)

    best_params = study.best_params
    et = ExtraTreesRegressor(**best_params)
    et.fit(X_train, y_train)
    y_pred = et.predict(X_test)

    print(f'cat is  :  {p}')
    plt.plot(y_test.values[0:100], "b")
    plt.plot(y_pred[0:100], "r--")
    plt.show()

    # Process test data
    test_data =data.iloc[-2*31*30+31*29:]
    globals()[p] = test_data[test_data['State'] == p]

    # Check if there is any data for this state
    if globals()[p].shape[0] == 0:
        print(f"No data available for state {p} in the test set.")
    else:
        print(f'prov is under test  :  {p}')
        globals()[p].info()


        output['date'] = globals()[p]['PersianDate']
        #output['y_actual'] = globals()[p]['portion']

        # Drop columns without using 'inplace=True' and check if data exists
        X_test = globals()[p].drop(columns=['portion', 'state_name_en', 'State','order_count'])
        if X_test.shape[0] == 0:
            print(f"No valid data for prediction in state {p}")
        else:
            output[p] = et.predict(X_test)
output['date'] = globals()[p]['PersianDate']
output.to_excel(f'result25+1.xlsx')


In [ ]:
models = {}
index = 62620
#index = 3954
train_data ,test_data = trainval_test_split_by_index(data, index)


output_df = pd.DataFrame(test_data['PersianDate'].unique(), columns=['PersianDate'])

i=0
for state in states:
  i += 1
  print(f" {i} / {len(states)}. {state} ... ", end='' )

  save = {}
  # State Segment Selection
  train = state_selection(train_data, state)
  train.fillna(train.mean())

  ## X and y preparation
  X_train1, X_test1, y_train1, y_test1 = serial_train_test_split(train, test_size=0.05)
  #X_train1, X_test1, y_train1, y_test1 = serial_train_test_split(train, test_size=0.0001)###################


  ## Scaling
  #X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, scaler, scaler_y = scale(X_train1, X_test1, y_train1, y_test1)
  X_train_scaled = X_train1
  X_test_scaled = X_test1
  y_train_scaled = y_train1
  y_test_scaled = y_test1


  ## Grid Search for Best Parameters
  #best_params = grid(X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled)


  ## Model Defenition
  model = XGBRegressor('''n_estimators=500, max_depth=best_params['max_depth'], learning_rate=0.1,
                       reg_lambda=best_params['reg_lambda'], reg_alpha=best_params['reg_alpha'],
                       min_child_weight=best_params['min_child_weight'], random_state=42,
                       subsample=best_params['subsample']''')


  # Fit the model on training data
  model.fit(X_train_scaled, y_train_scaled)


  # Prediting
  y_train_pred = model.predict(X_train_scaled)
  y_test_pred = model.predict(X_test_scaled)


  # Calculate RMSE and R2 Score for training and testing sets
  train_rmse = rmse(y_train_scaled, y_train_pred)
  test_rmse = rmse(y_test_scaled, y_test_pred)
  #train_score = model.score(X_train_scaled, y_train_scaled)
  # test_score = model.score(X_test_scaled, y_test_scaled)



  # Testing Phase
  test = state_selection(test_data, state)
  #X_scaled = scaler.transform(test)
  X_scaled = test
  predict = model.predict(X_scaled)



In [ ]:
  # y inversion
  #output = scaler_y.inverse_transform(predict.reshape(-1, 1))

  #output = outlier_normalization(output, y_train1)

  # SHAP EXPLAINER
  #explainer = shap.TreeExplainer(model)
  #explanation = explainer(X_scaled)

  shap_values = explanation.values
# make sure the SHAP values add up to marginal predictions
  np.abs(shap_values.sum(axis=1) + explanation.base_values - predict).max()
  ## Saving Phase
  save['model'] = model
  save['train_rmse'] = train_rmse
  save['test_rmse'] = test_rmse
  # save['train_score'] = train_score
  # save['test_score'] = test_score
  save['output'] = output
  save['X_train_scaled'] = X_train_scaled
  save['shap_values'] = shap_values
  save['explanation'] = explanation
  models[state] = save

  ## Creating output Dataframe
  output_df[state]= output
  print("DONE")


# output_df.to_excel( 'StatesAnalysis_output.xlsx', sheet_name='predictions')

# with open('StatesAnalysiModels.json', 'w') as fp:
#     json.dump(models, fp)

In [ ]:
normalized_df = output_df.copy()
sums = np.array(output_df.drop('PersianDate', axis=1).sum(axis=1)).reshape(-1,1)
normalized_df = normalized_df.loc[:, normalized_df.columns != 'PersianDate'] / sums
normalized_df = pd.concat([output_df['PersianDate'], normalized_df],axis=1)

In [ ]:
# write = 'StatesAnalysis_Danesh.xlsx'
write = 'StatesAnalysis_Shad.xlsx'

with pd.ExcelWriter(write) as writer:
    output_df.to_excel(writer, sheet_name='Predictions')
    normalized_df.to_excel(writer, sheet_name='Normalized')

In [ ]:
for province, configs in models.items():
# Create a summary plot with the bar plot type

  shap.summary_plot(configs["shap_values"], configs["X_train_scaled"], plot_type='bar', show=False)
  # Customize the plot appearance
  plt.title("SHAP Summary Plot - Feature Importance")
  plt.xlabel("SHAP Value", fontsize=12)
  plt.ylabel("Feature", fontsize=12)
  plt.xticks(fontsize=10)
  plt.yticks(fontsize=10)
  plt.tight_layout()
  plt.savefig(f'fresh_{province}.png')


In [ ]:
for province, configs in models.items():

  shap.plots.beeswarm(explanation)
  plt.savefig(f'fresh_{province}_swarm.png')